# SQL Query Propositions & Explanations - Mehtab Mahir

Note: There was usage of LLMs, specifically OpenAI's o3 mini

## 1. High-Value Customers (More Than 5 Orders in 2016)

**Explanation:** This query identifies customers who placed more than 5 orders in 2016. By counting the orders per customer and filtering out those with 5 or fewer orders, it highlights high-value customers who are likely more engaged and contribute significantly to overall sales.

In [ ]:
SELECT 
    o.CustomerID, 
    COUNT(o.OrderID) AS TotalOrders
FROM Sales.Orders AS o
WHERE YEAR(o.OrderDate) = 2016
GROUP BY o.CustomerID
HAVING COUNT(o.OrderID) > 5  
ORDER BY TotalOrders DESC;

## 2. Customers Who Placed the Most Orders in a Single Month (2016)

**Explanation:** This query aggregates order counts by customer and month for the year 2016. By grouping the data by both `CustomerID` and the month of the order, it pinpoints which customers had a particularly busy month, suggesting periods of elevated purchasing activity.

In [ ]:
SELECT 
    o.CustomerID, 
    MONTH(o.OrderDate) AS OrderMonth,
    COUNT(o.OrderID) AS OrdersInMonth
FROM Sales.Orders AS o
WHERE YEAR(o.OrderDate) = 2016
GROUP BY o.CustomerID, MONTH(o.OrderDate)
ORDER BY OrdersInMonth DESC;

## 3. Customers Who Placed Orders in At Least 6 Different Months (2015)

**Explanation:** This query calculates the number of distinct months in which each customer placed an order during 2015. It then filters for those with orders in more than 6 months, thereby identifying customers with consistent buying behavior throughout a significant portion of the year.

In [ ]:
SELECT 
    o.CustomerID, 
    COUNT(DISTINCT MONTH(o.OrderDate)) AS MonthsWithOrders
FROM Sales.Orders AS o
WHERE YEAR(o.OrderDate) = 2015
GROUP BY o.CustomerID
HAVING COUNT(DISTINCT MONTH(o.OrderDate)) > 6  
ORDER BY o.CustomerID;

## 4. Customers With the Highest Total Orders Value in 2016

**Explanation:** This query aggregates the total number of orders placed by each customer in 2016 and orders the results in descending order. Although it counts the number of orders (rather than the monetary value), it serves as a proxy for identifying customers who are purchasing the most overall, thereby reflecting a high total order value.

In [ ]:
SELECT 
    o.CustomerID, 
    COUNT(o.OrderID) AS TotalOrders
FROM Sales.Orders AS o
WHERE YEAR(o.OrderDate) = 2016
GROUP BY o.CustomerID
ORDER BY TotalOrders DESC;

## 5. Customers With the Highest Number of Orders on a Single Day in 2016

**Explanation:** This query first calculates the number of orders placed by each customer on each day in 2016. It then finds the maximum daily order count for each customer, revealing those who had days of exceptionally high activity—potentially indicating bulk orders or repeated purchases on a single day.

In [ ]:
SELECT 
    o.CustomerID, 
    MAX(OrderCount) AS MaxOrdersInOneDay
FROM ( 
    SELECT 
        CustomerID, 
        OrderDate, 
        COUNT(OrderID) AS OrderCount
    FROM Sales.Orders
    WHERE YEAR(OrderDate) = 2016
    GROUP BY CustomerID, OrderDate
) AS DailyOrders
GROUP BY CustomerID
ORDER BY MaxOrdersInOneDay DESC;

## 6. Customers With the Longest Time Between Orders in 2016

**Explanation:** This query computes the gap in days between the earliest and latest order dates for each customer during 2016. It highlights customers with the longest intervals between orders, which might indicate less frequent purchasing or significant changes in buying behavior throughout the year.

In [ ]:
SELECT 
    o.CustomerID, 
    DATEDIFF(DAY, MIN(o.OrderDate), MAX(o.OrderDate)) AS DaysBetweenFirstAndLastOrder
FROM Sales.Orders AS o
WHERE YEAR(o.OrderDate) = 2016
GROUP BY o.CustomerID
ORDER BY DaysBetweenFirstAndLastOrder DESC;

## 7. Most Common Order Date in 2016

**Explanation:** This query groups all orders by their date in 2016 and counts the number of orders for each date. Sorting the results in descending order by the count helps identify the specific day when the highest number of orders were placed, indicating peak ordering activity.

In [ ]:
SELECT 
    o.OrderDate, 
    COUNT(o.OrderID) AS OrderCount
FROM Sales.Orders AS o
WHERE YEAR(o.OrderDate) = 2016
GROUP BY o.OrderDate
ORDER BY OrderCount DESC;

## 8. Customers Who Placed Orders in Every Year

**Explanation:** This query checks each customer’s order history across all available years in the dataset. It compares the number of distinct years in which a customer has made an order to the total number of distinct years present in the Sales.Orders table. Customers meeting this criterion have been consistently active, placing at least one order in every year.

In [ ]:
SELECT CustomerID
FROM Sales.Orders
GROUP BY CustomerID
HAVING COUNT(DISTINCT YEAR(OrderDate)) = (SELECT COUNT(DISTINCT YEAR(OrderDate)) FROM Sales.Orders);

## 9. Largest Streak of Orders in Consecutive Days (2016)

**Explanation:** This multi-step query determines the longest sequence of consecutive order days for each customer in 2016. It first assigns a row number to each order and then uses the difference between two row numbers (one based on the order date and one adjusted by a constant reference date) to group orders into consecutive-day streaks. In subsequent steps, it calculates the length of each streak and ultimately selects the maximum streak per customer, ordering the results from longest to shortest streak.

In [ ]:
WITH RankedOrders AS ( 
    SELECT 
        CustomerID,
        OrderDate,
        ROW_NUMBER() OVER (PARTITION BY CustomerID ORDER BY OrderDate) - 
        ROW_NUMBER() OVER (PARTITION BY CustomerID ORDER BY DATEDIFF(DAY, '1900-01-01', OrderDate)) AS StreakGroup
    FROM Sales.Orders
    WHERE YEAR(OrderDate) = 2016
),
Streaks AS (
    SELECT 
        CustomerID, 
        COUNT(OrderDate) AS StreakLength
    FROM RankedOrders
    GROUP BY CustomerID, StreakGroup
)
SELECT CustomerID, MAX(StreakLength) AS MaxStreak
FROM Streaks
GROUP BY CustomerID
ORDER BY MaxStreak DESC;

## 10. First Order Date for Each Customer (2016)

**Explanation:** This query finds the first time each customer placed an order in 2016 by grouping the orders by `CustomerID` and selecting the earliest order date. Sorting these dates in ascending order helps in understanding the timeline of customer engagement and identifying when each customer first began ordering during the year.

In [ ]:
SELECT 
    o.CustomerID, 
    MIN(o.OrderDate) AS FirstOrderDate
FROM Sales.Orders AS o
WHERE YEAR(o.OrderDate) = 2016
GROUP BY o.CustomerID
ORDER BY FirstOrderDate;